In [1]:
import os
import ftplib
import pandas as pd
from glob import glob
import ahocorasick
from Bio import SeqIO

In [6]:
#Create local directories to place the downloaded files
pep_dir = 'pep_files'
try:
    os.mkdir(pep_dir)
except FileExistsError:
    pass

# Download protein files from the pan-genomes hub and put them under pep_dir 
base_url = 'ftp.ensembl.org'
samples_dir = '/pub/rapid-release/species/Homo_sapiens/'

samples_list = []
#Get list of samples
try:
    ftp = ftplib.FTP(base_url, timeout=3600)
    ftp.login()
    ftplib.FTP.maxline = 100000
    ftp.cwd(samples_dir)
    samples_list = ftp.nlst()
except ftplib.all_errors:
    print('Failed to connect to the ftp site')

print("Number of Samples available: " + str(len(samples_list)))

#get protein files from the ftp link
years = ['2022']
months = ['0'+str(x) if len(str(x))==1 else str(x) for x in range(0,13) ]
samples = []
file_downloaded = 0
for sample in samples_list:
    sample_genes_dir = '{id}/ensembl/geneset'.format(id=sample)
    
    ftp.cwd(sample_genes_dir)
    sample_genes_files = ftp.nlst()
    file_url, file_name = '', ''
    
    for year in years[::-1]:
        for month in months[::-1]:
            #try to get the latest month release
            if '{}_{}'.format(year, month) in sample_genes_files:
                file_url = '{}_{}/'.format(year, month)
                file_name_cdna = 'Homo_sapiens-{}-{}_{}-cdna.fa.gz'.format(sample, year, month)
                file_name_pep = 'Homo_sapiens-{}-{}_{}-pep.fa.gz'.format(sample, year, month)
                break

    try:
        pep_file = '{}/{}'.format(pep_dir, file_name_pep)
        #download pep files
        if not os.path.isfile(pep_file.replace('.gz', '')):
            print('Downloading: {}'.format(pep_file))
            ftp.retrbinary('RETR {}/{}'.format(file_url, file_name_pep), 
                     open(pep_file, 'wb').write)
            os.system('gunzip {}'.format(pep_file))

    except ftplib.error_perm as e:
        print('Skipped downloading for sample {}, error: {}'.format(sample, e))

    ftp.cwd(samples_dir)

print("Total #files in {} : {}".format(pep_dir, len(glob(pep_dir+"/*.fa"))))

Number of Samples completed: 97
Downloading: pep_files/Homo_sapiens-GCA_009914755.4-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018466835.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018466845.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018466855.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018466985.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018467005.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018467015.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018467155.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018467165.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018469405.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018469415.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018469425.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018469665.1-2022_07-pep.fa.gz
Downloading: pep_files/Homo_sapiens-GCA_018469675.1-2022_0

In [125]:
filtered_peps_input_file = 'gca_peptides_for_deeplc_95thperc_observations_ms2pip_filtered.tsv'
filtered_peps = [l.strip().split('\t') for l in open(filtered_peps_input_file).readlines()]
header = filtered_peps[0]
peps = filtered_peps[1::]

print(header)
print(len(peps))
pep_seqs = set([x[header.index('sequence_x')] for x in peps])


['sequence_x', 'protein_accessions', 'charge', 'scan_number', 'peptidoform', 'exp_mass_to_charge', 'calc_mass_to_charge', 'seq', 'tr', 'modifications', 'reference_file_name', "'OpenMS:Target-decoy PSM q-value'", 'Posterior error probability', 'sample_id', 'preds_tr', 'error', 'abserror', 'error_percentile', 'PeptideAtlas_observations', 'GPMDB_observations', 'Uniprot_Extended_PE', 'delta_mass', 'usi', 'number_misscleavages', 'pearsonr_B', 'pearsonr_Y', 'dot_product', 'count_B', 'count_Y', 'corrected_pearsonr_B', 'corrected_pearsonr_Y', 'corrected_dot_product', 'total_ions', 'number_peaks', 'signal_to_noise', 'diff_highest_lowest']
8553


In [23]:
#create automator for the peptides
auto = ahocorasick.Automaton()

for seq in pep_seqs:
    auto.add_word(seq, seq)

auto.make_automaton()

In [132]:
#search each protein file and report proteins that contain the peptides
peps_prots = {}
for prot_file in glob('pep_files/*.fa'):
    prot_seqs = set()
    prots_dict = {}
    parsed_file = SeqIO.parse(prot_file, 'fasta')
    sample_name = prot_file.split('/')[-1].split('-')[1]

    for record in parsed_file:
        try:
            prots_dict[str(record.seq)].append(sample_name + ' ' +  str(record.id) + ' ' + str(record.description))
        except KeyError:
            prots_dict[str(record.seq)] = [sample_name + ' ' + str(record.id) + ' ' + str(record.description)]
    print('Total number of unique protein sequences = {} in the fasta file: {}'.format(len(prots_dict.keys()), prot_file))

    #get non-canonical peptides that are found in canonical proteins from ensembl
    found_prots = set()
    
    for prot_seq in prots_dict.keys():
        for end_ind, found in auto.iter(prot_seq):
            found_prots.add(found)
            try:
                peps_prots[found].append(prots_dict[prot_seq])
            except KeyError:
                peps_prots[found] = [prots_dict[prot_seq]]

print('Numner of peptides found in the protein files {} and number of peptides {}'.format(len(peps_prots), len(peps)))


Total number of unique protein sequences = 86385 in the fasta file: pep_files/Homo_sapiens-GCA_018506155.1-2022_07-pep.fa
Total number of unique protein sequences = 86629 in the fasta file: pep_files/Homo_sapiens-GCA_018471075.1-2022_08-pep.fa
Total number of unique protein sequences = 89176 in the fasta file: pep_files/Homo_sapiens-GCA_018467165.1-2022_07-pep.fa
Total number of unique protein sequences = 89153 in the fasta file: pep_files/Homo_sapiens-GCA_018506975.1-2022_07-pep.fa
Total number of unique protein sequences = 86448 in the fasta file: pep_files/Homo_sapiens-GCA_018471555.1-2022_08-pep.fa
Total number of unique protein sequences = 86504 in the fasta file: pep_files/Homo_sapiens-GCA_018470425.1-2022_07-pep.fa
Total number of unique protein sequences = 89230 in the fasta file: pep_files/Homo_sapiens-GCA_018469425.1-2022_07-pep.fa
Total number of unique protein sequences = 89224 in the fasta file: pep_files/Homo_sapiens-GCA_018469925.1-2022_07-pep.fa
Total number of unique p

In [143]:
pep_gene_info = {}
sample_pep_info = {}
gene_pep_info = {}
prot_pep_info = {}
for pep in set([x[0] for x in filtered_peps[1::]]):
    prots = [x.split(' ') for xs in peps_prots[pep] for x in xs]
    samples, proteins, genes, gene_symbols, transcripts, gene_biotypes = set(), set(), set(), set(), set(), set()
    
    for prot in prots:
        samples.add(prot[0])
        proteins.add(prot[1])
        genes.add([x for x in prot if x.startswith('gene:')][0].split(':')[-1])
        transcripts.add([x for x in prot if x.startswith('transcript:')][0].split(':')[-1])
        gene_biotypes.add([x for x in prot if x.startswith('gene_biotype:')][0].split(':')[-1])
        
        try:
            gene_symbols.add([x for x in prot if x.startswith('gene_symbol:')][0].split(':')[-1])
        except IndexError:
            pass

    for sample in samples:
        try:
            sample_pep_info[sample].add(pep)
        except KeyError:
            sample_pep_info[sample] = set(pep)
    
    for gene in genes:
        try:
            gene_pep_info[gene].append(pep)
        except KeyError:
            gene_pep_info[gene] = [pep]

    for prot_id in proteins:
        try:
            prot_pep_info[prot_id].append(pep)
        except KeyError:
            prot_pep_info[prot_id] = [pep]
    
    pep_gene_info[pep] = [','.join(gene_symbols), ','.join(gene_biotypes), ','.join(samples), ','.join(proteins), ','.join(genes), ','.join(transcripts)]
    


In [144]:
with open('peps_all_info.tsv', 'w') as peps_out:
    
    peps_out.write('\t'.join(header) + '\t' + '\t'.join(['gene_symbols', 'gene_biotypes', 'samples', 'proteins', 'genes', 'transcripts']) + '\n')
    for pep in filtered_peps[1::]:
        try:
            peps_out.write('\t'.join(pep) + '\t' + '\t'.join(pep_gene_info[pep[0]])+'\n')
        except KeyError:
            print('{} not found in any prot file'.format(pep[0]))
